In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)

In [ ]:
test1 = 'a'+'b'
print(test1)

In [2]:
from model.pipeline.preparation import prepare_data, split_zero_dataframe, compute_cumulative_sum
from model.pipeline.model import (
    get_train_test, 
    build_parallel_model, 
    build_local_regression_model, 
    build_local_moving_average, 
    build_intermittent_model,
    build_auto_arima_model, 
    merge_forecast_dataframe, 
    evaluate_model, 
    get_best_forecast,
    search_space_regression,
    initial_train_size,
    build_global_regression_model,
    build_local_regression_model_Nixtla,
    build_global_regression_model_nixtla,
    my_init_config
)
import pandas as pd
import numpy as np
import time

/root/miniconda3/envs/bda_forecast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Y_df = pd.read_csv('M09_customer.csv')
Y_df

,yyyymmdd,office_wwcode,sub_mcode,object_code,size_code,total_customer,total_home,total_gov,total_small,total_state,total_big,total_ind,total_fix
0,20181130,5541032,2490234301,1320,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20161130,5542032,2490333402,1210,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20121130,5542017,2490317101,1600,9,9.0,0.0,3.0,0.0,0.0,6.0,0.0,0.0
3,20150430,5532028,2390830403,1310,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20170430,5522028,2290728101,1210,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301678,20210531,5552016,2590517401,1320,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3301679,20200531,5552021,2590530101,1320,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3301680,20210731,5511016,2190916401,1210,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3301681,20200131,5521022,2290622101,1320,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Y_df1210 = Y_df[Y_df['object_code'] == 1210]

In [5]:
# เลือกคอลัมน์ที่ต้องการ sum
columns_to_sum = ['total_big', 'total_fix', 'total_gov', 'total_home', 'total_ind', 'total_small', 'total_state']

# ทำการ group by ตาม 'office_wwcode', 'ds', 'size_code' และ sum ค่า
Y_df1210 = Y_df1210.groupby(['office_wwcode', 'yyyymmdd', 'sub_mcode'])[columns_to_sum].sum().reset_index()

In [6]:
SELECT_COLUMNS = [col for col in Y_df1210.columns if col != 'total_customer' or col != 'size_code']
CATEGORY_COLUMNS = ["office_wwcode", "sub_mcode", "customer_type"]
DATETIME_COLUMN = "yyyymmdd"
MELT_ID = ['office_wwcode','sub_mcode', 'ds']

In [ ]:
start_time = time.time()
Y_df1210 = prepare_data(
    dataframe=Y_df1210, 
    select_columns=SELECT_COLUMNS,
    category_columns=CATEGORY_COLUMNS, 
    datetime_column=DATETIME_COLUMN, 
    melt_id=MELT_ID
)

Y_df1210 = Y_df1210[Y_df1210['ds'].dt.year >= 2015]

nonzero_df1210, mixzero_df1210 = split_zero_dataframe(Y_df1210)
all_df1210 = pd.concat([nonzero_df1210, mixzero_df1210])

cum_nonzero_df1210 = compute_cumulative_sum(nonzero_df1210)
cum_mixzero_df1210 = compute_cumulative_sum(mixzero_df1210)
cum_all_df1210 = compute_cumulative_sum(all_df1210)

train_all_df1210, test_all_df1210 = get_train_test(all_df1210)
train_nonzero_df1210, test_nonzero_df1210 = get_train_test(nonzero_df1210)
train_mixzero_df1210, test_mixzero_df1210 = get_train_test(mixzero_df1210)

train_cum_all_df1210, test_cum_all_df1210 = get_train_test(cum_all_df1210)
train_cum_nonzero_df1210, test_cum_nonzero_df1210 = get_train_test(cum_nonzero_df1210)
train_cum_mixzero_df1210, test_cum_mixzero_df1210 = get_train_test(cum_mixzero_df1210)

inter_non_accum_forecast = build_intermittent_model(training_data=train_all_df1210)
arima_non_accum_forecast = build_auto_arima_model(training_data=train_all_df1210, is_accumulated=False)
arima_accum_forecast = build_auto_arima_model(training_data=train_cum_all_df1210, is_accumulated=True)
global_non_accum_all_regression_forecast = build_global_regression_model_nixtla(training_data=train_all_df1210, testing_data=test_all_df1210, is_accumulated=False)
global_non_accum_nonzero_regression_forecast = build_global_regression_model_nixtla(training_data=train_nonzero_df1210, testing_data=test_nonzero_df1210, is_accumulated=False)
global_non_accum_mixzero_regression_forecast = build_global_regression_model_nixtla(training_data=train_mixzero_df1210, testing_data=test_mixzero_df1210, is_accumulated=False)
global_accum_all_regresion_forecast = build_global_regression_model_nixtla(training_data=train_cum_all_df1210, testing_data=test_cum_all_df1210, is_accumulated=True)
global_accum_nonzero_regresion_forecast = build_global_regression_model_nixtla(training_data=train_cum_nonzero_df1210, testing_data=test_cum_nonzero_df1210, is_accumulated=True)
global_accum_mixzero_regresion_forecast = build_global_regression_model_nixtla(training_data=train_cum_mixzero_df1210, testing_data=test_cum_mixzero_df1210, is_accumulated=True)
local_non_accum_regression_forecast = build_parallel_model(
    training_data=train_all_df1210, 
    testing_data=test_all_df1210, 
    build_model_func=build_local_regression_model_Nixtla, 
    is_accumulated=False
)
local_accum_regression_forecast = build_parallel_model(
    training_data=train_cum_all_df1210, 
    testing_data=test_cum_all_df1210, 
    build_model_func=build_local_regression_model_Nixtla, 
    is_accumulated=True
)
moving_non_accum_forecast = build_parallel_model(
    training_data=train_all_df1210, 
    testing_data=test_all_df1210, 
    build_model_func=build_local_moving_average, 
    is_accumulated=False
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"use time: {elapsed_time:.5f} seconds")

In [ ]:
merge_df = merge_forecast_dataframe(
    inter_non_accum_forecast, 
    arima_non_accum_forecast, 
    arima_accum_forecast, 
    global_non_accum_all_regression_forecast, 
    global_non_accum_nonzero_regression_forecast, 
    global_non_accum_mixzero_regression_forecast,
    global_accum_all_regresion_forecast,
    global_accum_nonzero_regresion_forecast,
    global_accum_mixzero_regresion_forecast,
    local_non_accum_regression_forecast,
    local_accum_regression_forecast,
    moving_non_accum_forecast
)
evaluate_df = evaluate_model(merge_df)
best_forecast = get_best_forecast(merge_df, evaluate_df)